# Introduction to Natural Language Processing 2 Lab04

## Introduction

Your company wants to sell a moderation API tackling toxic content on Twitter. They ask you to come up with a model which detect toxic tweets. You remember your NLP classes, and start looking for existing models or datasets, and find a collection of [academic Twitter dataset on HuggingFace hub](https://huggingface.co/datasets/tweet_eval). Especially, the `hate` and `offensive` datasets seem close to what you are looking for.

In [3]:
# Get the datasets from HuggingFace
from datasets import load_dataset

hate_dataset = load_dataset("tweet_eval", "hate")
offensive_dataset = load_dataset("tweet_eval", "offensive")

train_hate = hate_dataset["train"]
valid_hate = hate_dataset["validation"]
test_hate = hate_dataset["test"]
train_offensive = offensive_dataset["train"]
valid_offensive = offensive_dataset["validation"]
test_offensive = offensive_dataset["test"]

Found cached dataset tweet_eval (/home/coartix/.cache/huggingface/datasets/tweet_eval/hate/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


  0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/11916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/860 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1324 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /home/coartix/.cache/huggingface/datasets/tweet_eval/offensive/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### 1. (1 point) Pick one of the datasets between `hate` and `offensive`, and justify your choice. Remember that it is for a commercial application (there is a good and a bad answer).

The model needs to detect toxic tweets.

In [13]:
from IPython.display import Markdown, display

# Get a preview of the data where the label is 1
display(Markdown("### Hate dataset"))
display(train_hate.filter(lambda example: example["label"] == 1).select(range(5))["text"])
display(Markdown("### Offensive dataset"))
display(train_offensive.filter(lambda example: example["label"] == 1).select(range(5))["text"])

### Hate dataset

Loading cached processed dataset at /home/coartix/.cache/huggingface/datasets/tweet_eval/hate/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-279708674ccb8ecd.arrow


['A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎',
 '@user @user real talk do you have eyes or were they gouged out by a rapefugee?',
 'your girlfriend lookin at me like a groupie in this bitch!',
 "I AM NOT GOING AFTER YOUR EX BF YOU LIEING SACK OF SHIT ! I'm done with you dude that's why I dumped your ass cause your a lieing 😂😡 bitch",
 'Send home migrants not in need of protection, Peter Dutton tells UN, HEY DUTTON HOW ABOUT THE ONES THAT HAVE STAYED AND NOT LEFT THE COUNTRY WHEN THEY SHOULD OVERSTAYERS ? WHY DONT YOU GO AND ROUND ALL THEM UP ?']

### Offensive dataset

Loading cached processed dataset at /home/coartix/.cache/huggingface/datasets/tweet_eval/offensive/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-ad72cdd5a8672614.arrow


['@user Eight years the republicans denied obama’s picks. Breitbarters outrage is as phony as their fake president.',
 "@user She has become a parody unto herself? She has certainly taken some heat for being such an....well idiot. Could be optic too  Who know with Liberals  They're all optics.  No substance",
 '@user Your looking more like a plant #maga #walkaway',
 '@user Antifa would burn a Conservatives house down and CNN would be there lighting the torches &amp; throwing gas on the flames.',
 '@user They cite Jones being banned for violating Twitter\'s ToS. There are blue checkmarks spewing the same, if not worse, kind of shit. If you are going to play the anyone can get banned"" card. Shouldn\'t these people also receive bans and suspensions? #VerifiedHate""']

From the preview of the two datasets we can see that the `hate` dataset tackles more toxic messages than the `offensive` one. We will then pick the `hate` dataset.

## Evaluating the dataset

Let's start with a data analysis.  
### 1. (1 point) Describe the dataset. Look at the splits, proportion of classes, and see what you can figure out by just looking at the text.

In [24]:
import pandas as pd

# Get proportion of splits and classes
print("Training split:", len(train_hate), "\n" + str(pd.Series(train_hate["label"]).value_counts()),
      "\nValidation split:", len(valid_hate), "\n" + str(pd.Series(valid_hate["label"]).value_counts()),
        "\nTest split:", len(test_hate), "\n" + str(pd.Series(test_hate["label"]).value_counts()))

Training split: 9000 
0    5217
1    3783
dtype: int64 
Validation split: 1000 
0    573
1    427
dtype: int64 
Test split: 2970 
0    1718
1    1252
dtype: int64


From the proportion of classes we can see that there are more non-hateful tweets than hateful ones. Roughly 58% of the data on each split is non-hateful.

0.5784511784511784